# Population activity

In [1]:
from imp import reload
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import rcParams
import sys, os
import time
sys.path.append(os.path.abspath('../presentation')) # include path with style
sys.path.append(os.path.abspath('../simulation/')) # include path with simulation specifications
import pres_style as style; reload(style)
import helper_functions as hf; reload(hf)

# Import specific moduls
import model_class; reload(model_class)
import sim_params as sim; reload(sim)
%matplotlib inline

show_fig = False
save_fig = True
xfactor = 2.6
style.rcParams['figure.figsize'] = (xfactor*6.2, xfactor*3.83) 
figure_path = os.path.join(".", "figures")

reverse_order = True # do analysis such that plots resemble those of the paper (starting with L6i)

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3agg.py:18: UserWarning: The Gtk3Agg backend is known to not work on Python 3.x with pycairo. Try installing cairocffi.
  "The Gtk3Agg backend is known to not work on Python 3.x with pycairo. "
/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 2 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)


### Basic data

In [3]:
# DATA
sim_spec = "a1.0_t300.2_00"

data_file = "micro"
data_sup_path = "/export/data-schuessler/data_microcircuit/"
data_path = os.path.join(data_sup_path, data_file)
res_file_name = sim_spec + "_res.hdf5"
path_res_file = os.path.join(data_path, res_file_name)

## Spike counts and power spectrum

In [10]:
from scipy.signal import savgol_filter as sav
def rebin(array, n_zoom=2):
    """Rebinning of an array without interpolation."""
    overlap = array.shape[-1] % n_zoom
    if overlap != 0:
        pre_rebin = array[:, :-overlap].copy() # cut off the end
    else:
        pre_rebin = array.copy()

    rebinned = pre_rebin[:, ::n_zoom]
    for k in range(1, n_zoom):
        rebinned += pre_rebin[:, k::n_zoom]

    return rebinned, overlap

# Basic data
(area, t_sim, t_trans, t_measure, dt, 
        populations, layers, types, 
        n_populations, n_layers, n_types) = hf.basic_data(path_res_file)
sim_spec2 = "0"

# Open file: results
with h5py.File(path_res_file, "r") as res_file:
    res_grp = res_file[sim_spec2]
    n_rec_spikes = res_grp["n_rec_spikes"][:]
    hist_spikes  = res_grp["hist_spikes"][:]
    hist_times   = res_grp["hist_times"][:]
    normed_hist = (hist_spikes.T / n_rec_spikes).T

fig = plt.figure(figsize=(xfactor*6.2, 2*xfactor*3.83) )
ax0 = hf.add_subplot(fig, n_rows_cols=(1, 2), index_row_col=(0, 0), rowspan=1, colspan=1)
ax1 = hf.add_subplot(fig, n_rows_cols=(1, 2), index_row_col=(0, 1), rowspan=1, colspan=1)

# Rebinning
time_bin_size = 3.e-3 # s; time step after rebinning
time_step = dt * 1.e-3 # s; original time step
n_zoom = int(time_bin_size / time_step) # factor by which to zoom out
reb_hist, overlap = rebin(normed_hist, n_zoom=n_zoom)
stretch = np.floor(1. / np.max(reb_hist))  # stretch each histogram...  

for i, population in enumerate(populations):
    if overlap == 0:
        reb_hist_times = hist_times[::n_zoom]
    else:
        reb_hist_times = hist_times[:-overlap][::n_zoom]
    if reverse_order:
        y_values = stretch * reb_hist[i] + n_populations - (i + 1)
    else:
        y_values = stretch * reb_hist[i] + i
    ax0.plot(reb_hist_times, y_values, 
             "-", alpha=0.8, color=colors[i], drawstyle="steps")
    ax0.plot(hist_times[[0, -1]], [i]*2, "--", color="gray")
    
    # Power spectrum
    glob_ac = reb_hist[i]
    ga_mean = np.mean(glob_ac)

    ps = np.abs(np.fft.fft(glob_ac - ga_mean))**2
    freqs = np.fft.fftfreq(glob_ac.size, time_step * n_zoom)
    idx = np.argsort(freqs)

    # Filtered
    filtered = sav(ps[idx], 301, 5, deriv=0, delta=1.0, axis=-1, mode='interp', cval=0.0)
    
    # Normalize
    y_max = 3 * np.max(filtered)
    ps_norm = ps[idx] / y_max
    ps_norm[ps_norm > 1] = 1
    filtered_norm = filtered / y_max   
    
    ax1.plot(freqs[idx], ps_norm + i, "-", alpha=0.2, linewidth=1, color=colors[i])
    ax1.plot(freqs[idx], filtered_norm + i, "-", alpha=0.8, color=colors[i])    
    ax1.plot(freqs[idx][[0, -1]], [i]*2, "--", color="gray")
    
xlim = (0.2, 1.2)    

# Normalized pop activity ticks
ax0.set_xlim(*xlim)
ax0.set_xlabel('simulation time / s')
ax0.set_ylim(0, n_populations)
ax0.set_ylabel('Normalized population activity / $\\frac{spikes}{%.1f \,\mathrm{ms}}$'%(0.1*n_zoom))
# Axes labels
n_ticks = 5
yto           = np.arange(0, 1 - 1. / n_ticks, 1. / n_ticks)
yt_offset     = np.repeat(np.arange(0, n_populations), n_ticks-1)
y_ticks = np.tile(yto, n_populations) + yt_offset
y_ticklabels = np.tile(np.arange(0, stretch * (1 - 1. / n_ticks), stretch / n_ticks), n_populations)
ax0.set_yticks(y_ticks)
ax0.set_yticklabels(y_ticklabels)
ax0.grid(False)

# No ticks!
ax1.set_xlabel('f / Hz')
xlim = (0, 200)
ax1.set_xlim(*xlim)
ax1.set_yticks([])
ax1.set_ylabel("Power spectral density")
ax1.set_ylim(0, n_populations)
ax1.grid(False)

# Population labels
ax2 = ax1.twinx()
ax2.set_ylabel('Population')
ax2.set_yticks(np.arange(0.5, 8, 1))
yticklabels = hf.rlbl(populations)
if reverse_order:
    yticklabels = yticklabels[::-1]
ax2.set_yticklabels(yticklabels)
ax2.set_ylim(0, n_populations)

for ax in fig.axes:
    style.fixticks(ax)
    
fig_name = "population_activity"
if save_fig:
    fig.savefig(os.path.join(figure_path, fig_name + ".pdf"), format="pdf")
    fig.savefig(os.path.join(figure_path, fig_name + ".png"), format="png")